# P04: Diffusion & Spectral Graph Theory (PhD Level)

## 1. Introduction: Hearing the Shape of a City

In modules M8/M9, we treated diffusion as a process $x(t)$. But on a complex urban network, diffusion reveals the **topology** itself.
Kac's famous question "Can one hear the shape of a drum?" translates to: "Can observing random walkers tell us the structural bottlenecks of a city?"

In this PhD-level notebook, we move beyond simple random walks to **Spectral Graph Theory**. We will show that the **Eigenvalues of the Graph Laplacian** govern the timescale of diffusion, and the **Eigenvectors (Fiedler Vector)** reveal the natural community structure (e.g., distinguishing "River North" from "River South" in a city).

### Core Mappings
| Urban Network Analysis | Quantum/Statistical Physics |
| :--- | :--- |
| Adjacency Matrix $A$ | Hamiltonian (Interaction) |
| Laplacian $L = D - A$ | Kinetic Energy Operator $-\nabla^2$ |
| Fiedler Value $\lambda_2$ | Energy Gap (Mass Gap) |
| Fiedler Vector $v_2$ | First Excited State Wavefunction |
| Commute Time | Green's Function (Propagator) |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy import linalg

# Set seed
np.random.seed(42)

## 2. Constructing a City with a Bottleneck

We construct a "Barbell Graph": Two dense communities (Urban Centers) connected by a narrow bridge (Bridge/Tunnel).
This is a classic test case for spectral clustering.

In [ ]:
# Create Barbell Graph: two complete graphs of size m connected by path of length len
m = 20
G = nx.barbell_graph(m1=m, m2=5) 
# m1: size of bells, m2: length of bridge

pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(8, 6))
nx.draw(G, pos, node_size=50, node_color='gray', edge_color='gray', alpha=0.6)
plt.title(f"Synthetic City: Two Centers connected by a Bridge (N={len(G)}) ")
plt.show()

## 3. The Laplacian and Diffusion Dynamics

The diffusion equation on a graph is:
$$ \frac{d\mathbf{p}}{dt} = - L \mathbf{p} $$
where $L = D - A$ is the Graph Laplacian.
The solution is $\mathbf{p}(t) = e^{-Lt} \mathbf{p}(0)$.

Expanding in eigenvectors $\phi_k$ of $L$ ($L\phi_k = \lambda_k \phi_k$):
$$ \mathbf{p}(t) = \sum_k c_k e^{-\lambda_k t} \phi_k $$

- $\lambda_0 = 0$: Steady state (uniform if regular).
- $\lambda_1$: The smallest non-zero eigenvalue (**Fiedler Value**). It dictates the **slowest decay mode** toward equilibrium.

**Physics Interpretation**: $\lambda_1$ is the "Energy Gap". If the gap is small, relaxation is slow (Critical Slowing Down). This happens exactly when the city has a bottleneck.

In [ ]:
# Compute Laplacian
L = nx.laplacian_matrix(G).toarray()

# Eigendecomposition
# eigenvalues w, eigenvectors v
w, v = linalg.eigh(L)

# Sort just in case (eigh usually returns sorted)
idx = np.argsort(w)
w = w[idx]
v = v[:, idx]

print("First 5 Eigenvalues:", w[:5])
lambda_2 = w[1]
print(f"Fiedler Value (Algebraic Connectivity): {lambda_2:.4f}")
print("-> A small value indicates the graph is easy to 'cut' (bottleneck exists).")

## 4. The Fiedler Vector (Community Detection)

The eigenvector corresponding to $\lambda_1$ is called the **Fiedler Vector**. 
Its sign ($+$ or $-$) effectively partitions the graph into two weakly connected components.

Let's color the nodes by their Fiedler vector value.

In [ ]:
fiedler_vec = v[:, 1]

plt.figure(figsize=(10, 6))
nx.draw(G, pos, 
        node_color=fiedler_vec, 
        cmap=plt.cm.coolwarm, 
        node_size=100, 
        edge_color='gray')
plt.title(f"Nodes colored by Fiedler Vector $\phi_2$ ($\lambda_2={lambda_2:.3f}$)")
plt.colorbar(label='Fiedler Vector Component')
plt.show()

## 5. Diffusion Simulation

Let's simulate the diffusion process $e^{-Lt}$. Instead of running random walkers, we compute the matrix exponential directly (continuum limit).
We start a pulse of mass at one end (Node 0) and see how it leaks through the bottleneck.

In [ ]:
p0 = np.zeros(len(G))
p0[0] = 1.0 # Start at node 0 (left bell)

times = [0, 10, 50, 200]

plt.figure(figsize=(15, 4))

for i, t in enumerate(times):
    pt = linalg.expm(-L * t) @ p0
    
    plt.subplot(1, 4, i+1)
    nx.draw(G, pos, 
            node_color=pt, 
            cmap=plt.cm.YlOrRd, 
            node_size=80, 
            # vmin=0, vmax=0.1, # Fixed scale to see spreading
            edge_color='lightgray')
    plt.title(f"t={t}")

plt.suptitle("Time Evolution of Probability Density", y=1.05)
plt.tight_layout()
plt.show()